# Klayout integration

In gdsfactory code driven flow you define your components, circuits and reticles in python or YAML code.

For rapid iteration, gdsfactory comes with a Klayout macro `klive` that runs inside klayout, so that when you run `component.show()` in python, it displays your GDS in Klayout.

![workflow](https://i.imgur.com/yquTcM7.png)



You have 2 ways to install the klayout gdsfactory plugin to be able to see live updates on your GDS files:

- from the terminal by typing `gf tool install` after installing gdsfactory `pip install gdsfactory`
- using klayout package manager (see image below), Tools --> Manage Packages

![klayout package](https://i.imgur.com/AkfcCms.png)

What does the klayout integration installs?

- Klive: connects python to klayout, to show GDS files directly, when running `component.show()`
- generic layermap: for the generic gdsfactory technology.
- generic DRC: for generic gdsfactory technology


## Klayout DRC

You can write DRC rule decks from gdsfactory and customize the shortcut to run the DRC macro in Klayout.

Here are some rules explained in [repo generic DRC technology](https://github.com/klayoutmatthias/si4all) and [video](https://peertube.f-si.org/videos/watch/addc77a0-8ac7-4742-b7fb-7d24360ceb97)

![rules1](https://i.imgur.com/gNP5Npn.png)

In [1]:
import gdsfactory as gf
from gdsfactory.geometry.write_drc import (
    write_drc_deck_macro,
    rule_enclosing,
    rule_width,
    rule_space,
    rule_separation,
    rule_density,
    rule_area,
)

2022-11-07 00:10:42.683 | INFO     | gdsfactory.config:<module>:45 - Load '/home/runner/work/gdsfactory/gdsfactory/gdsfactory' 5.54.0


In [2]:
help(write_drc_deck_macro)

Help on function write_drc_deck_macro in module gdsfactory.geometry.write_drc:

write_drc_deck_macro(rules: List[str], layers: Dict[str, Tuple[int, int]], name: str = 'generic', filepath: Union[str, pathlib.Path, NoneType] = None, shortcut: str = 'Ctrl+Shift+D', mode: str = 'tiled', threads: int = 4, tile_size: int = 500, tile_borders: Optional[int] = None, **kwargs) -> str
    Write klayout DRC macro.
    
    You can customize the shortcut to run the DRC macro from the Klayout GUI.
    
    Args:
        rules: list of rules.
        layers: layer definitions can be dict or dataclass.
        name: drc rule deck name.
        filepath: Optional macro path (defaults to .klayout/drc/name.lydrc).
        shortcut: to run macro from klayout GUI.
        mode: tiled, default or deep (hiearchical).
        threads: number of threads.
        tile_size: in um for tile mode.
        tile_borders: sides for each. Defaults None to automatic.
    
    modes:
    
    - default
        - flat po

In [3]:
rules = [
    rule_width(layer="WG", value=0.2),
    rule_space(layer="WG", value=0.2),
    rule_width(layer="M1", value=1),
    rule_width(layer="M2", value=2),
    rule_space(layer="M2", value=2),
    rule_separation(layer1="HEATER", layer2="M1", value=1.0),
    rule_enclosing(layer1="M1", layer2="VIAC", value=0.2),
    rule_area(layer="WG", min_area_um2=0.05),
    rule_density(
        layer="WG", layer_floorplan="FLOORPLAN", min_density=0.5, max_density=0.6
    ),
]

drc_rule_deck = write_drc_deck_macro(
    rules=rules,
    layers=gf.LAYER,
    shortcut="Ctrl+Shift+D",
)

2022-11-07 00:10:43.546 | INFO     | gdsfactory.geometry.write_drc:write_drc_deck_macro:302 - Wrote DRC deck to '/home/runner/.klayout/drc/generic.lydrc' with shortcut 'Ctrl+Shift+D'


## Klayout connectivity checks

Thanks to [SiEPIC-Tools](https://github.com/SiEPIC/SiEPIC-Tools) klayout macro gdsfactory supports component overlap and connectivity checks.

This is enabled by default for any components that use the `strip` cross_section, thanks to having `add_pins=add_pins_siepic` and `add_bbox=add_bbox_siepic` by default.

![Siepic](https://i.imgur.com/wHnWxMb.png)